In [1]:
# -*- coding: utf-8 -*-
"""ProgramacionLineal.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1YNvg0HIpwytq1vTjDZbkyBXyEypesh7Y
"""

!pip install pulp

import pandas as np
from pulp import *
from pandas import DataFrame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 108.3 MB/s eta 0:00:00


In [2]:
df = DataFrame({'Alimentos':['maiz','frijoles','trigo','soja','alfalfa'],'Proteina':[10, 20, 15, 35, 17],
                'Calcio':[0.5, 1.2, 0.3, 0.4, 1.8],'Calorias':[3300, 3100, 3000, 4000, 2700],'Costo':[1200, 2500, 1800, 3000, 2200]})


In [3]:
df.head()

,Alimentos,Proteina,Calcio,Calorias,Costo
0,maiz,10,0.5,3300,1200
1,frijoles,20,1.2,3100,2500
2,trigo,15,0.3,3000,1800
3,soja,35,0.4,4000,3000
4,alfalfa,17,1.8,2700,2200


In [4]:
#### Creamos un problema LP con LpProblem, usamos LpMinimize pues queremos minimizar la función con los costos
problema = LpProblem("ProblemaDieta",LpMaximize)


In [5]:
### Creamos una lista con los alimentos
alimentos = list(df['Alimentos'])


In [6]:
### Creamos un diccionario con los costos
costos = dict(zip(alimentos,df['Costo']))

In [7]:
### Creamos un diccionario con las calorias
calorias = dict(zip(alimentos,df['Calorias']))

In [8]:
### Creamos un diccionario con los valores de calcio
calcio = dict(zip(alimentos,df['Calcio']))

In [9]:
### Creamos un diccionario con los valores de proteína
proteina = dict(zip(alimentos,df['Proteina']))

In [10]:
### Ponemos 0 ya que queremos valores >= 0 y 'Continuous' para regresernos valores reales o bien 'Integer' para valores enteros
alimentos_vars = LpVariable.dicts("Alimentos",alimentos,0,cat='Integer')

In [11]:
### Función objetivo
problema += lpSum([costos[i]*alimentos_vars[i] for i in alimentos]), "Costo total de la dieta"

In [12]:
### Establecemos las restricciones de Calorías
problema += lpSum([calorias[f] * alimentos_vars[f] for f in alimentos]) >= 2000.0, "MínimoCalorías"
problema += lpSum([calorias[f] * alimentos_vars[f] for f in alimentos]) <= 2200.0, "MáximoCalorías"

In [13]:
### Restricciones de proteínas
problema += lpSum([proteina[f] * alimentos_vars[f] for f in alimentos]) >= 158.4, "MínimoProteinas"
problema += lpSum([proteina[f] * alimentos_vars[f] for f in alimentos]) <= 258.4, "MáximoProteinas"

In [14]:
### Restricciones de Calcio
problema += lpSum([calcio[f] * alimentos_vars[f] for f in alimentos]) >= 1000.0, "MinimoCalcio"
problema += lpSum([calcio[f] * alimentos_vars[f] for f in alimentos]) <= 2500.0, "MaximoCalcio"


In [15]:
### Escribimos el problema en un archivo .lp
problema.writeLP("SimpleDietProblem.lp")

[Alimentos_alfalfa,
 Alimentos_frijoles,
 Alimentos_maiz,
 Alimentos_soja,
 Alimentos_trigo]

In [16]:
### PuLP resuelve el problema con las restricciones dadas y la función objetivo.
problema.solve()

-1

In [17]:
### Veamos si existe una solución y si ésta es óptima
print("Status:", LpStatus[problema.status])

Status: Infeasible


In [18]:
### Imprimimos la dieta balanceada óptima
print("La dieta balanceada óptima (menor costo) para las gallinas ponedoras, donde cada unidad equivale a 1 kg, consiste en:\n" + "-" * 110)
for alimento in alimentos:
    cantidad = alimentos_vars[alimento].varValue
    print(alimento, "=", round(cantidad, 2), "kg")

La dieta balanceada óptima (menor costo) para las gallinas ponedoras, donde cada unidad equivale a 1 kg, consiste en:
--------------------------------------------------------------------------------------------------------------
maiz = 0.0 kg
frijoles = 0.0 kg
trigo = 0.0 kg
soja = 0.0 kg
alfalfa = 0.81 kg


In [19]:
### Imprimimos el costo
print("El costo diario de la dieta es: ${}".format(round(value(problema.objective),2)))

El costo diario de la dieta es: $1792.59
